THIS MODEL HAS BEEN MADE AND RUN ON GOOGLE COLAB

install the python sdk

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.5 MB/s eta 0:00:00


Import generative ai package

In [ ]:
import google.generativeai as genai

In [ ]:
# Used to securely store your API key
from google.colab import userdata

Setup your API key

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

Instantiate the model

In [ ]:
text_model = genai.GenerativeModel('gemini-pro')

When an LLM chatbot gives an answer to a question, the last numerical value is it's final answer (based on observation). So make a function which extracts the last numerical value

In [ ]:
import re

def extract_last_number(input_string):
    # Split the string using regex
    parts = re.split(r'[^a-zA-Z0-9.]', input_string)

    # Initialize a variable to store the last number found
    last_number = None

    # Iterate over each part in reverse order
    for part in reversed(parts):
        # Attempt to convert the part to an integer or float
        try:
            last_number = int(part)
            break
        except ValueError:
            try:
                last_number = float(part)
                break
            except ValueError:
                pass  # If the part is not a number, continue to the next part

    return last_number


Make a function which takes the question as an input, use prompts which will make the AI use a chain-of-thoughts method, then extract the numerical result. It will not block the question for any false-positive harm category instance.

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold
def solve_question(question):
  # Construct Chain-of-Thought prompt
  prompt1 = f"""
  Use the Chain-of-Thought methodology to explain your step-by-step calculations or logic.
  Q: {question}
  Let's think step by step.
  """
  p=text_model.generate_content(prompt1,
                                safety_settings={
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }).text
  #print(p)
  answer=extract_last_number(p)
  if(answer==int(answer)):
    answer=int(answer)
  return answer


sometimes AI may generate a result which may not have a numeric value, or a value which overflows the assigned data range. so error handling is done

In [ ]:
def solve(question):
  try:
    return solve_question(question)
  except (KeyError, TypeError, ValueError, OverflowError) as e:
    print("Error Handled ")
    return solve(question)

since this model takes an extreme amount of time, the csv file containing the questions were split into two parts and run in each code cell for predicting answers

In [ ]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv('TestSet.csv')

# Split the DataFrame into two parts of 249 rows each
part1 = df.iloc[:249]
part2 = df.iloc[249:]

# Write each part to separate CSV files
part1.to_csv('part1.csv', index=False)
part2.to_csv('part2.csv', index=False)


predicting answers for part 1

In [ ]:
# Read the input CSV file
input_df1 = pd.read_csv('part1.csv')

# Iterate over each row in the input DataFrame
output_data = []
for index, row in input_df1.iterrows():
    # Get the question from the current row
    question = row['QUESTION']

    # Call the solve function to get the answer
    #solve(question)
    answer = solve(question)

    # Append the ID and answer to the output data list
    output_data.append({'ID': row['ID'], 'LABEL': answer})
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
Error Handled 
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248


predicting answers for part 2

In [ ]:
input_df2 = pd.read_csv('part2.csv')

# Iterate over each row in the input DataFrame
for index, row in input_df2.iterrows():
    # Get the question from the current row
    question = row['QUESTION']

    # Call the solve function to get the answer
    #solve(question)
    answer = solve(question)
    if(answer==int(answer)):
      answer=int(answer)

    # Append the ID and answer to the output data list
    output_data.append({'ID': row['ID'], 'LABEL': answer})
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1091.43ms


59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
Error Handled 
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
Error Handled 
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


creating an output file for evaluation

In [ ]:
# Create a DataFrame from the output data
output_df = pd.DataFrame(output_data)

# Write the output DataFrame to a CSV file
output_df.to_csv('zero_shot.csv', index=False)